In [1]:
from lxml import etree
import urllib.request as urllib2
import pandas as pd
import numpy as np
import requests
import lxml.html as html
from datetime import datetime, date, timedelta
import json
import time
import re
from sklearn.linear_model import LinearRegression, LogisticRegression

Содержание датафрейма:
Rec_date - дата получения прогноза
Resrc - ресурсы, с которого загружен прогноз
Forec_date - дата и время, на которые составлен прогноз
Temp_l, Temp_h - температура нижняя и верхняя граница
Clouds - облачность 0 - ясно, 1- малооблачно, 2 - облачно, 3 - пасмурно
Presp - осадки 4 - дождь, 5 - ливень, 6,7 – снег, 8 - гроза, 9 - нет данных, 10 - без осадков
Prob_presp - интенсивность осадков, если есть 0 - возможен дождь/снег, 1 - дождь/снег
Prob_rage - вероятность грозы 0 - возможна гроза, 1 - гроза
Wind_l, Wind_h - скорость ветра (Минимальная и максимальная)
Wind_dir - направление ветра (0 - северный, 1 - северо-восточный итд)

Общий вид и стрктура xml:
<MMWEATHER>
<REPORT type="frc3">
<TOWN index="37" sname="%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0" latitude="56" longitude="38">
<FORECAST day="03" month="08" year="2016" hour="09" tod="1" predict="0" weekday="4">
<PHENOMENA cloudiness="3" precipitation="4" rpower="0" spower="1"/>
<PRESSURE max="755" min="753"/>
<TEMPERATURE max="26" min="20"/>
<WIND min="1" max="2" direction="5"/>
<RELWET max="87" min="62"/>
<HEAT min="22" max="22"/>
</FORECAST>
<FORECAST day="03" month="08" year="2016" hour="15" tod="2" predict="0" weekday="4">
<PHENOMENA cloudiness="3" precipitation="4" rpower="1" spower="1"/>
<PRESSURE max="755" min="753"/>
<TEMPERATURE max="26" min="21"/>
<WIND min="2" max="2" direction="7"/>
<RELWET max="87" min="62"/>
<HEAT min="23" max="23"/>
</FORECAST>
<FORECAST day="03" month="08" year="2016" hour="21" tod="3" predict="0" weekday="4">
<PHENOMENA cloudiness="3" precipitation="10" rpower="0" spower="0"/>
<PRESSURE max="755" min="755"/>
<TEMPERATURE max="21" min="18"/>
<WIND min="2" max="2" direction="7"/>
<RELWET max="88" min="87"/>
<HEAT min="19" max="19"/>
</FORECAST>
<FORECAST day="04" month="08" year="2016" hour="03" tod="0" predict="0" weekday="5">
<PHENOMENA cloudiness="3" precipitation="10" rpower="0" spower="0"/>
<PRESSURE max="757" min="755"/>
<TEMPERATURE max="17" min="16"/>
<WIND min="1" max="1" direction="7"/>
<RELWET max="88" min="85"/>
<HEAT min="16" max="16"/>
</FORECAST>
</TOWN>
</REPORT>
</MMWEATHER>
TOWN - информация о пункте прогнозирования:
index -	уникальный код города
sname	- закодированное название города
latitude -	широта в целых градусах
longitude -	долгота в целых градусах
FORECAST - информация о сроке прогнозирования:
day, month, year -	дата, на которую составлен прогноз в данном блоке
hour -	местное время, на которое составлен прогноз
tod -	время суток, для которого составлен прогноз: 0 - ночь 1 - утро, 2 - день, 3 - вечер
weekday -	день недели, 1 - воскресенье, 2 - понедельник, и т.д.
predict -	заблаговременность прогноза в часах
PHENOMENA - атмосферные явления:
cloudiness -	облачность по градациям: 0 - ясно, 1- малооблачно, 2 - облачно, 3 - пасмурно
precipitation -	тип осадков: 4 - дождь, 5 - ливень, 6,7 – снег, 8 - гроза, 9 - нет данных, 10 - без осадков
rpower -	интенсивность осадков, если они есть. 0 - возможен дождь/снег, 1 - дождь/снег
spower -	вероятность грозы, если прогнозируется: 0 - возможна гроза, 1 - гроза
PRESSURE -	атмосферное давление, в мм.рт.ст.
TEMPERATURE - температура воздуха, в градусах Цельсия
WIND -	приземный ветер
min, max -	минимальное и максимальное значения средней скорости ветра, без порывов (м/с)
direction - направление ветра в румбах, 0 - северный, 1 - северо-восточный, и т.д.
RELWET -	относительная влажность воздуха, в %
HEAT - комфорт - температура воздуха по ощущению одетого по сезону человека, выходящего на улицу

In [38]:
#Функция высчитывает значение комфортности на основании Температуры (Цельсий), скорости ветра (м/с) и Влажности(%) и возвращает комфортность
def Comfort(Temperature,  Wind, Humid):
    t = Temperature
    V = Wind
    F = 100-Humid
    return t*(1-0.003*F)-0.385*(V**0.59)*((36.6-t)+0.622*(V-1))+((0.0015*V+0.0008)*(36.6-t)-0.0167)*F

#Функция пытается прочитать csv файл с данным о погоде по названию. В случае отсутствия выводится ошибка
def or_data(name = 'wh_data.csv'):
    try:
        wh_data = pd.read_csv(name)
        wh_data.Time = wh_data.Time.apply(lambda x: int(x))
        return wh_data
    except:
        raise Exception('Is not created')

#Функция загружает и парсит датафрейм с сайта. scl - конфигурация сдвига шкалы, для будущего обучения
#Возвращает датафрейм с загруженным данными и колонками Time(в формате YYYYMMDDHH), Temp_1 Temp_2 комфортности и DP_1 вероятность осадков и их тип
def wrk_with_openwheather(url = "http://xml.meteoservice.ru/export/gismeteo/point/37.xml", scl = 30):
    url = urllib2.urlopen(url)
    tree = etree.fromstring(url.read())
    wh_data = pd.DataFrame(columns=['Time','Temp_1','Temp_2','DP_1'])
    for child in tree.iter('FORECAST'):
        string_date = str(child.attrib['year'])+str(int(child.attrib['month']))+str(int(child.attrib['day']))+str(int(child.attrib['hour']))
        for predcom in child:
            if predcom.tag=='PHENOMENA':
                flurries_snow = int(predcom.attrib['precipitation'])
                fl_sn_power = int(predcom.attrib['rpower'])
            if predcom.tag=='TEMPERATURE':
                temp_max = predcom.attrib['max']
                temp_min = predcom.attrib['min']
            if predcom.tag=='WIND':
                wind_max = predcom.attrib['max']
                wind_min = predcom.attrib['min']
            if predcom.tag=='RELWET':
                hum_max = predcom.attrib['max']
                hum_min = predcom.attrib['min']
        if flurries_snow==4 or flurries_snow==5 or flurries_snow==8:
            downpoor=1
        elif flurries_snow==6 or flurries_snow==7:
            downpoor=-1
        else:
            downpoor=0
        if fl_sn_power==0:
            coeff=0.5
        else:
            coeff=1    
        wh_data.loc[len(wh_data)]=[string_date, Comfort(int(temp_max), float(wind_max), int(hum_max))+scl,
                                   Comfort(int(temp_min), int(wind_min), int(hum_min))+scl, downpoor*coeff]
    return wh_data

#Аналогично предыдущему, но грузит с другого сайта
def wrk_with_underweather(url = "http://api.wunderground.com/api/c4d5d9d99bd4ed54/hourly/q/Russia/Moscow.json", scl=30):
    req = requests.get(url)
    pred_for_use = json.loads(req.text)
    wh_data = pd.DataFrame(columns=['Time','Temp_3','DP_2'])
    pred_for_wrk = pred_for_use['hourly_forecast']
    for i in range(len(pred_for_wrk)):
        if int(pred_for_wrk[i]['FCTTIME']['hour'])==3 or int(pred_for_wrk[i]['FCTTIME']['hour'])==9 or int(pred_for_wrk[i]['FCTTIME']['hour'])==15 or int(pred_for_wrk[i]['FCTTIME']['hour'])==21:
            child = pred_for_wrk[i]['FCTTIME']
            string_date = str(child['year'])+str(int(child['mon']))+str(int(child['mday']))+str(int(child['hour']))
            temp = int(pred_for_wrk[i]['temp']['metric'])
            hum = int(pred_for_wrk[i]['humidity'])
            coeff = float(pred_for_wrk[i]['pop'])/100
            wind = float(pred_for_wrk[i]['wspd']['metric'])/3.6
            if pred_for_wrk[i]['qpf']['metric']!=0:
                downpoor = 1
            elif pred_for_wrk[i]['snow']!=0:
                downpoor = -1
            else:
                downpoor = 0
            wh_data.loc[len(wh_data)] = [string_date, Comfort(int(temp), float(wind), int(hum))+scl, downpoor*coeff]
    return wh_data

#Функция обрабатывает исторические данные и возвращает датафрейм с ними
def wrk_with_history(scl=30):
    yesterday = str(date.today() - timedelta(1))
    req = requests.get("http://api.wunderground.com/api/c4d5d9d99bd4ed54/history_"+yesterday[0:4]+yesterday[5:7]+yesterday[8:10]+"/q/Russia/Moscow.json")
    pred_for_use = json.loads(req.text)
    wh_h_data = pd.DataFrame(columns=['Time','Temp_h','DP_h'])
    pred_for_wrk = pred_for_use['history']['observations']
    for i in range(len(pred_for_wrk)):
        obj = pred_for_wrk[i]
        if int(obj['date']['min'])==0 and (int(obj['date']['hour'])==3 or int(obj['date']['hour'])==9 or int(obj['date']['hour'])==15 or int(obj['date']['hour'])==21):
            temp = float(obj['tempm'])
            hum = float(obj['hum'])
            wind = float(obj['wspdm'])/3.6
            rain = int(obj['rain'])
            snow = -int(obj['snow'])
            string_date=int(str(int(yesterday[0:4]))+str(int(yesterday[5:7]))+str(int(yesterday[8:10]))+str(int(obj['date']['hour'])))
            if rain!=0:
                dwpr=1
            elif snow!=0:
                dwpr=-1
            else:
                dwpr=0
            wh_h_data.loc[len(wh_h_data)] = [string_date, Comfort(temp,wind,hum)+scl,dwpr]
    return wh_h_data

#Функция предназначена для объединения датафрейов с предсказаниями по принципу минимума и по дате
def merger_frame(data_1, data_2):
    return data_1.merge(data_2, how='inner', on='Time')

#Функция предназначена для объединения датафреймов с историческими данными, опускается отсутствие истор данных, можно упразднить
def merger_hist(data_1, data_2):
    return data_1.merge(data_2, how='left', on='Time')

#Функция записывает датафрейм в csv файл
def to_csv_writter(data, name = 'wh_data.csv'):
    data.to_csv(name, index=False)

#Функция объединяет датафреймы по оси строк, предназначена для работы со старыми и новыми файлами
def prepar_for_wrt(data_old, data_new):
    return pd.concat([data_old, data_new], ignore_index=True)

#Функция создает линейные регрессионные модели для предсказания температуры и возвращает их
def leaners(frame_for_teach):
    frame_for_teach = frame_for_teach.dropna(axis=0)
    lst_dp = []
    for x in frame_for_teach.columns:
        if re.search(r'DP_\d', x)!=None:
            lst_dp.append(x)
    lst_dp.append('DP_h')
    frame_dp = frame_for_teach[lst_dp]
    lst_dp.append('Time')
    frame_tmp = frame_for_teach.drop(lst_dp, axis=1)
    
    tmp_labels = frame_tmp['Temp_h']
    tmp_train = frame_tmp.drop(['Temp_h'], axis=1)
    dp_labels = frame_dp['DP_h']
    dp_train = frame_dp.drop(['DP_h'], axis=1)
    
    return LinearRegression().fit(tmp_train, tmp_labels), LogisticRegression().fit(dp_train, dp_labels)

#Функция возвращает предсказанные значения комфортности и осадков
def predictor(clf_tmp, clf_dp, data_new):
    data_new = data_new.drop(['Time'], axis=1)
    lst_dp = []
    for x in frame_for_teach.columns:
        if re.search(r'DP_\d', x)!=None:
            lst_dp.append(x)
    frame_dp = data_new[lst_dp]
    frame_tmp = data_new.drop(lst_dp, axis=1)
    
    return clf_tmp.predict(frame_tmp), clf_dp.predict(frame_dp)



In [39]:
#Блок загрузки
lst = [wrk_with_openwheather(), wrk_with_underweather()]
res = merger_frame(lst[0],lst[1])
if len(lst)>2:
    for i in range(2,len(lst)):
        res = merger_frame(res,lst[i])
data_h = wrk_with_history()
try:
    data_old_frc = or_data()
    data_fin = prepar_for_wrt(data_old_frc, res)
except Exception as e:
    print(e)
    data_fin=res
try:
    data_old_h = or_data('wh_data_h.csv')
    data_fin_h = prepar_for_wrt(data_old_h, data_h)
except Exception as e:
    data_fin_h = data_h
    print(e)
to_csv_writter(data_fin)
to_csv_writter(data_fin_h, 'wh_data_h.csv')
try:
    data_f_wrk = merger_hist(data_fin, data_fin_h)
    clf_t, clf_d = leaners(data_f_wrk)
    forecast_t, forecast_d = predictor(clf_t, clf_d, res)
except:
    print("В обучении что-то не так")

In [37]:
frame_for_teach = data_f_wrk
frame_for_teach = frame_for_teach.dropna(axis=0)
lst_dp = []
for x in frame_for_teach.columns:
    if re.search(r'DP_\d', x)!=None:
        lst_dp.append(x)
lst_dp.append('DP_h')
frame_dp = frame_for_teach[lst_dp]
lst_dp.append('Time')
frame_tmp = frame_for_teach.drop(lst_dp, axis=1)
    
tmp_labels = frame_tmp['Temp_h']
tmp_train = frame_tmp.drop(['Temp_h'], axis=1)
dp_labels = frame_dp['DP_h']
dp_train = frame_dp.drop(['DP_h'], axis=1)
clf_1 = LinearRegression().fit(tmp_train, tmp_labels)
clf_2 = LogisticRegression().fit(dp_train, dp_labels)

In [40]:
forecast_t

array([ 18.62173948,  18.32724385,  19.93327513,  17.63571011])

In [6]:
data_fin_h

,Time,Temp_h,DP_h
0,2016953,30.705467,0
1,2016959,28.500226,1
2,20169515,28.500226,1
3,20169521,23.441197,0
4,2016963,20.068510,0
5,2016969,18.722810,0
6,20169615,30.697913,0
7,20169621,29.744407,0
8,2016973,20.068510,0
9,2016979,19.563890,0


In [11]:
dt = data_fin.merge(data_fin_h, how='outer', on='Time')
dt[dt.Time=='20169715']

,Time,Temp_1,Temp_2,DP_1,Temp_3,DP_2,Temp_h,DP_h
21,20169715,NaN,NaN,NaN,NaN,NaN,21.84972,0


In [36]:
#main


,Time,GisMeteo_min,GisMeteo_max,Gismeteo_dp,Underwh_temp,Inderwh_dp,Temp,Dwpr


In [6]:
data_fin

,Time,Temp_1,Temp_2,DP_1,Temp_3,DP_2
0,20169521,36.527740,35.204920,0.5,27.645104,0.00
1,2016963,32.548400,29.874840,0.0,22.678110,0.09
2,2016969,30.142089,30.236840,0.0,20.670804,0.06
3,20169615,32.849772,34.298947,0.0,28.725144,0.00


In [7]:
data_fin_h

,Time,Temp_h,DP_h
0,2016943,32.575560,0
1,2016949,21.414164,0
2,20169415,23.227684,1
3,20169421,26.447403,0


In [15]:
data_old_frc

,Time,Temp_1,Temp_2,DP_1,Temp_3,DP_2
0,2016973,32.587200,30.063080,0,25.922353,0.01
1,2016979,32.198532,31.547920,0,24.558187,0.02
2,20169715,34.479296,31.876809,0,30.041714,0.35


In [16]:
res

,Time,Temp_1,Temp_2,DP_1,Temp_3,DP_2
0,20169721,31.298569,31.387529,0,27.870324,0.19
1,2016983,30.320009,32.386847,0,21.721893,0.07
2,2016989,31.981452,32.549887,0,19.249695,0.15
3,20169815,33.977456,30.294746,0,22.176327,0.14


In [20]:
pd.concat([data_old_frc,res], ignore_index=True)

,Time,Temp_1,Temp_2,DP_1,Temp_3,DP_2
0,2016973,32.587200,30.063080,0,25.922353,0.01
1,2016979,32.198532,31.547920,0,24.558187,0.02
2,20169715,34.479296,31.876809,0,30.041714,0.35
3,20169721,31.298569,31.387529,0,27.870324,0.19
4,2016983,30.320009,32.386847,0,21.721893,0.07
5,2016989,31.981452,32.549887,0,19.249695,0.15
6,20169815,33.977456,30.294746,0,22.176327,0.14
